# 1. Data preprocessing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -Uqq fastbook
import fastbook
#fastbook.setup_book()

In [ ]:
from fastbook import *
from IPython.display import display,HTML
import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import random
import argparse

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize.treebank import TreebankWordDetokenizer

from torch.utils.data import DataLoader, TensorDataset

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
dataset = pd.read_excel('/content/drive/MyDrive/2022/NLP/train-dataset.xlsx')
dataset[:10]

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(dataset, test_size=0.2)
train.info()

train.to_excel('/content/drive/MyDrive/2022/NLP/train-dataset.xlsx', index=False)
test.to_excel('/content/drive/MyDrive/2022/NLP/test-dataset.xlsx', index=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9 entries, 8 to 6
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Story   9 non-null      object
dtypes: object(1)
memory usage: 144.0+ bytes


In [ ]:
train_df = pd.read_excel('/content/drive/MyDrive/2022/NLP/train-dataset.xlsx')
test_df = pd.read_excel('/content/drive/MyDrive/2022/NLP/test-dataset.xlsx')
train_df[:10]

In [ ]:
train_df['Story']

In [ ]:
def clean_text_data(data):
  data = str(data)
  data = re.sub('\Dn', '', data)
  data = re.sub('\S*@\S*\s?', '', data)
  data = re.sub('\s+', ' ', data)
  data = re.sub("\'", "", data)
  data = re.sub('\[.*?\]', '', data)
  data = re.sub('[%s]' % re.escape(string.punctuation), ' ', data)
  data = re.sub(r'\w*\d+\w*', '', data)
  data = re.sub(r'\s{2,}', ' ', data)
  data = re.sub(r"^\s+|\s+$", "", data) 
  return data

In [ ]:
def remove_stopwords(data):
  data = word_tokenize(data)
  stopwords = ['CHAPTER', '*','I','II','III','IV','V','VI','VII','VIII','IX','X','XI','XII','XIV','XV','XVI','XVII',
               'XVIII','XIX','XX','XXI','XXII','XXIII','XXIV','XXV','XXVI','XXVII','XXVIII','XXIX','XXX','XXXI','XXXII',
               'XXXII','XXXIV','XXXV','Chapter','n','“','’','”']

  data = [word for word in data if word not in stopwords]

  return data

In [ ]:
count = 0
for i in range(len(train_df['Story'])):
  txt = train_df['Story'][i]
  result = len(txt.split())
  count += result

count

52542

In [ ]:
clean_train_set =''
for i in range(len(train_df['Story'])):
  cleaned_txt = clean_text_data(train_df['Story'][i])
  clean_train_set += cleaned_txt

clean_train_set

'CHAPTER I WHICH TREATS OF THE CHARACTER AND PURSUITS OF THE FAMOUS GENTLEMAN DON QUIXOTE OF LA MANCHA CHAPTER II WHICH TREATS OF THE FIRST SALLY THE INGENIOUS DON QUIXOTE MADE FROM HOME CHAPTER III WHEREIN IS RELATED THE DROLL WAY IN WHICH DON QUIXOTE HAD HIMSELF DUBBED A KNIGHT CHAPTER IV OF WHAT HAPPENED TO OUR KNIGHT WHEN HE LEFT THE INN CHAPTER V IN WHICH THE NARRATIVE OF OUR KNIGHT’S MISHAP IS CONTINUED CHAPTER VI OF THE DIVERTING AND IMPORTANT SCRUTINY WHICH THE CURATE AND THE BARBER MADE IN THE LIBRARY OF OUR INGENIOUS GENTLEMAN CHAPTER VII OF THE SECOND SALLY OF OUR WORTHY KNIGHT DON QUIXOTE OF LA MANCHA CHAPTER VIII OF THE GOOD FORTUNE WHICH THE VALIANT DON QUIXOTE HAD IN THE TERRIBLE AND UNDREAMT OF ADVENTURE OF THE WINDMILLS WITH OTHER OCCURRENCES WORTHY TO BE FITLY RECORDED CHAPTER IX IN WHICH IS CONCLUDED AND FINISHED THE TERRIFIC BATTLE BETWEEN THE GALLANT BISCAYAN AND THE VALIANT MANCHEGAN CHAPTER X OF THE PLEASANT DISCOURSE THAT PASSED BETWEEN DON QUIXOTE AND HIS SQUIR

In [ ]:
clean_test_set =''
for i in range(len(test_df['Story'])):
  cleaned_txt = clean_text_data(test_df['Story'][i])
  clean_test_set += cleaned_txt

clean_test_set

'There were three of us—Mary Eliza d myself I was approachg fifte Mary was about a year yoger d Eliza betwe twelve d thirte years of age Mamma treated us all as childr d was bld to the fact that I waso lger what I had be Althoughot tall for my age or outwardly prestg a mly appearce my passis were awakg d the distctive feature of my sex although repose it looked maifict ough was very sufficitly developed wh der the fluce of feme excitemt As yet I had absolutelyo owledge of the uses of the differt orgs of sex My sisters d I all slept the same room They together e bed I ale other Who e was prest we had oft mutually examed the differt formatis of our sexes We had discovered that mutual hdlgs gave a certa amot of pleasg ssati d latterly my eldest sister had discovered that the hoodg d hoodg of my doodle as she called it sttly caused it to swell up d stiff as hard as a piece of wood My feelg of her little pky slit gave rise her toice ssatis but the slightest attempt to sert ev my fger the pa

In [ ]:
remove_stopwords_train_set = remove_stopwords(clean_train_set)
remove_stopwords_train_set

In [ ]:
remove_stopwords_test_set = remove_stopwords(clean_test_set)
remove_stopwords_test_set

In [ ]:
print(len(remove_stopwords_train_set), len(remove_stopwords_test_set))

49896 16207


In [ ]:
train_set = ''

for i in range(len(remove_stopwords_train_set)):
  word = remove_stopwords_train_set[i].lower()
  train_set += ' ' + word

train_set

' which treats of the character and pursuits of the famous gentleman don quixote of la mancha which treats of the first sally the ingenious don quixote made from home wherein is related the droll way in which don quixote had himself dubbed a knight of what happened to our knight when he left the inn in which the narrative of our knight s mishap is continued of the diverting and important scrutiny which the curate and the barber made in the library of our ingenious gentleman of the second sally of our worthy knight don quixote of la mancha of the good fortune which the valiant don quixote had in the terrible and undreamt of adventure of the windmills with other occurrences worthy to be fitly recorded in which is concluded and finished the terrific battle between the gallant biscayan and the valiant manchegan of the pleasant discourse that passed between don quixote and his squire sancho panza of what befell don quixote with certain goatherds of what a goatherd related to those with don 

In [ ]:
text_file = open('/content/drive/MyDrive/2022/NLP/train-set.txt', "w")
text_file.write(train_set)
text_file.close()

In [ ]:
train_df = pd.DataFrame({"Story":[train_set]})
train_df.to_csv('/content/drive/MyDrive/2022/NLP/train-data.csv')
train_df

,Story
0,which treats of the character and pursuits of the famous gentleman don quixote of la mancha which treats of the first sally the ingenious don quixote made from home wherein is related the droll way in which don quixote had himself dubbed a knight of what happened to our knight when he left the inn in which the narrative of our knight s mishap is continued of the diverting and important scrutiny which the curate and the barber made in the library of our ingenious gentleman of the second sally of our worthy knight don quixote of la mancha of the good fortune which the valiant don quixote ha...


In [ ]:
test_set = ''
for i in range(len(remove_stopwords_test_set)):
  word = remove_stopwords_test_set[i].lower()
  test_set += ' ' + word

test_set

' there were three of us—mary eliza d myself was approachg fifte mary was about a year yoger d eliza betwe twelve d thirte years of age mamma treated us all as childr d was bld to the fact that waso lger what had be althoughot tall for my age or outwardly prestg a mly appearce my passis were awakg d the distctive feature of my sex although repose it looked maifict ough was very sufficitly developed wh der the fluce of feme excitemt as yet had absolutelyo owledge of the uses of the differt orgs of sex my sisters d all slept the same room they together e bed ale other who e was prest we had oft mutually examed the differt formatis of our sexes we had discovered that mutual hdlgs gave a certa amot of pleasg ssati d latterly my eldest sister had discovered that the hoodg d hoodg of my doodle as she called it sttly caused it to swell up d stiff as hard as a piece of wood my feelg of her little pky slit gave rise her toice ssatis but the slightest attempt to sert ev my fger the pa was too gr

In [ ]:
text_file = open('/content/drive/MyDrive/2022/NLP/test-set.txt', "w")
text_file.write(test_set)
text_file.close()

In [ ]:
test_df = pd.DataFrame({"Story":[test_set]})
test_df.to_csv('/content/drive/MyDrive/2022/NLP/test-data.csv')

# 2. LSTM Model

In [ ]:
train_df = train_set
#train_df = pd.DataFrame(train_df, columns['Story'])
#text = train_df.str.cat(sep=' ')
#return text.split(' ')

In [ ]:
import torch
from torch import nn

class Model(nn.Module):
    def __init__(self, dataset):
        super(Model, self).__init__()
        self.lstm_size = 128
        self.embedding_dim = 128
        self.num_layers = 3

        n_vocab = len(dataset.uniq_words)
        self.embedding = nn.Embedding(
            num_embeddings=n_vocab,
            embedding_dim=self.embedding_dim,
        )
        self.lstm = nn.LSTM(
            input_size=self.lstm_size,
            hidden_size=self.lstm_size,
            num_layers=self.num_layers,
            dropout=0.2,
        )
        self.fc = nn.Linear(self.lstm_size, n_vocab)

    def forward(self, x, prev_state):
        embed = self.embedding(x)
        output, state = self.lstm(embed, prev_state)
        logits = self.fc(output)
        return logits, state

    def init_state(self, sequence_length):
        return (torch.zeros(self.num_layers, sequence_length, self.lstm_size),
                torch.zeros(self.num_layers, sequence_length, self.lstm_size))

In [ ]:
import torch
import pandas as pd
from collections import Counter

class Dataset(torch.utils.data.Dataset):
    def __init__(
        self,
        args,
    ):
        self.args = args
        self.words = self.load_words()
        self.uniq_words = self.get_uniq_words()

        self.index_to_word = {index: word for index, word in enumerate(self.uniq_words)}
        self.word_to_index = {word: index for index, word in enumerate(self.uniq_words)}

        self.words_indexes = [self.word_to_index[w] for w in self.words]

    def load_words(self):
        train_df = pd.read_csv('/content/drive/MyDrive/2022/NLP/train-data.csv')
        text = train_df['Story'].str.cat(sep=' ')
        return text.split(' ')

    def get_uniq_words(self):
        word_counts = Counter(self.words)
        return sorted(word_counts, key=word_counts.get, reverse=True)

    def __len__(self):
        return len(self.words_indexes) - self.args.sequence_length

    def __getitem__(self, index):
        return (
            torch.tensor(self.words_indexes[index:index+self.args.sequence_length]),
            torch.tensor(self.words_indexes[index+1:index+self.args.sequence_length+1]),
        )

In [ ]:
import easydict
import torch
import numpy as np
from torch import nn, optim
from torch.utils.data import DataLoader

def train(dataset, model, args):
    model.train()

    dataloader = DataLoader(dataset, batch_size=args.batch_size)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(args.max_epochs):
        state_h, state_c = model.init_state(args.sequence_length)

        for batch, (x, y) in enumerate(dataloader):
            optimizer.zero_grad()

            y_pred, (state_h, state_c) = model(x, (state_h, state_c))
            loss = criterion(y_pred.transpose(1, 2), y)

            state_h = state_h.detach()
            state_c = state_c.detach()

            loss.backward()
            optimizer.step()

            print({ 'epoch': epoch, 'batch': batch, 'loss': loss.item() })

In [ ]:
def predict(dataset, model, text, next_words=100):
    model.eval()

    words = [word.lower() for word in text.split(' ')]
    state_h, state_c = model.init_state(len(words))

    for i in range(0, next_words):
        x = torch.tensor([[dataset.word_to_index[w] for w in words[i:]]])
        y_pred, (state_h, state_c) = model(x, (state_h, state_c))

        last_word_logits = y_pred[0][-1]
        p = torch.nn.functional.softmax(last_word_logits, dim=0).detach().numpy()
        word_index = np.random.choice(len(last_word_logits), p=p)
        words.append(dataset.index_to_word[word_index])

    return words

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--max-epochs', type=int, default=10)
parser.add_argument('--batch-size', type=int, default=256)
parser.add_argument('--sequence-length', type=int, default=4)
args = parser.parse_args(args=[])

dataset = Dataset(args)
model = Model(dataset)

train(dataset, model, args)

{'epoch': 0, 'batch': 0, 'loss': 8.907578468322754}
{'epoch': 0, 'batch': 1, 'loss': 8.896855354309082}
{'epoch': 0, 'batch': 2, 'loss': 8.880342483520508}
{'epoch': 0, 'batch': 3, 'loss': 8.869119644165039}
{'epoch': 0, 'batch': 4, 'loss': 8.842793464660645}
{'epoch': 0, 'batch': 5, 'loss': 8.829748153686523}
{'epoch': 0, 'batch': 6, 'loss': 8.796198844909668}
{'epoch': 0, 'batch': 7, 'loss': 8.765440940856934}
{'epoch': 0, 'batch': 8, 'loss': 8.695627212524414}
{'epoch': 0, 'batch': 9, 'loss': 8.723553657531738}
{'epoch': 0, 'batch': 10, 'loss': 8.59627914428711}
{'epoch': 0, 'batch': 11, 'loss': 8.478372573852539}
{'epoch': 0, 'batch': 12, 'loss': 8.34405517578125}
{'epoch': 0, 'batch': 13, 'loss': 8.266641616821289}
{'epoch': 0, 'batch': 14, 'loss': 8.222445487976074}
{'epoch': 0, 'batch': 15, 'loss': 7.842077732086182}
{'epoch': 0, 'batch': 16, 'loss': 7.806699752807617}
{'epoch': 0, 'batch': 17, 'loss': 7.768446445465088}
{'epoch': 0, 'batch': 18, 'loss': 7.676513671875}
{'epoch'

In [ ]:
write_sentence = " ".join(predict(dataset, model, text='she is'))

print(write_sentence)

she is mey breathg th late toe the wedged sg pooh the so step would d predicted to to see his into s tut took were the scourged in for the word took related s haw first all he write and is called d ability his fe oak spot is about this july walkg op this rather ofter la pooh said was the usual momt that the momt himself cnot christopher xiously quite mor therefore them of you oh looked back a leks have protagist piglet somethg for he d face to pooh have um lg well piglet he no each with


# 3. Fine Tuning with GPT2

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [ ]:
!git clone https://github.com/huggingface/transformers.git

Cloning into 'transformers'...
remote: Enumerating objects: 97225, done.
remote: Counting objects: 100% (287/287), done.
remote: Compressing objects: 100% (176/176), done.
remote: Total 97225 (delta 136), reused 212 (delta 95), pack-reused 96938
Receiving objects: 100% (97225/97225), 90.35 MiB | 30.13 MiB/s, done.
Resolving deltas: 100% (71461/71461), done.


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def read_txt(txt_path):
  with open(txt_path, 'r') as f:
    txt_string = f.readlines()
  return txt_string

In [ ]:
weights_dir = "output"

In [ ]:
file_name = "/content/drive/MyDrive/2022/NLP/train-set.txt"
corpus_string = read_txt(file_name)

In [ ]:
corpus_string

[' which treats of the character and pursuits of the famous gentleman don quixote of la mancha which treats of the first sally the ingenious don quixote made from home wherein is related the droll way in which don quixote had himself dubbed a knight of what happened to our knight when he left the inn in which the narrative of our knight s mishap is continued of the diverting and important scrutiny which the curate and the barber made in the library of our ingenious gentleman of the second sally of our worthy knight don quixote of la mancha of the good fortune which the valiant don quixote had in the terrible and undreamt of adventure of the windmills with other occurrences worthy to be fitly recorded in which is concluded and finished the terrific battle between the gallant biscayan and the valiant manchegan of the pleasant discourse that passed between don quixote and his squire sancho panza of what befell don quixote with certain goatherds of what a goatherd related to those with don

In [ ]:
cmd = '''
python transformers/examples/pytorch/language-modeling/run_clm.py \
    --model_name_or_path gpt2 \
    --train_file {0} \
    --do_train \
    --num_train_epochs 3 \
    --overwrite_output_dir \
    --per_device_train_batch_size 2 \
    --output_dir {1}
'''.format(file_name, weights_dir)

In [ ]:
!{cmd}

06/07/2022 14:07:05 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
06/07/2022 14:07:05 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

def get_model_tokenizer(weights_dir, device = 'cuda'):
    print("Loading Model ...")
    model = GPT2LMHeadModel.from_pretrained(weights_dir)
    model.to('cuda')
    print("Model Loaded ...")
    tokenizer = GPT2Tokenizer.from_pretrained(weights_dir)
    return model, tokenizer

def generate_messages(
    model,
    tokenizer,
    prompt_text,
    stop_token,
    length,
    num_return_sequences,
    temperature = 0.7,
    k=20,
    p=0.3,
    repetition_penalty = 1.0,
    device = 'cuda'
):

    MAX_LENGTH = int(10000)
    def adjust_length_to_model(length, max_sequence_length):
        if length < 0 and max_sequence_length > 0:
            length = max_sequence_length
        elif 0 < max_sequence_length < length:
            length = max_sequence_length  # No generation bigger than model size
        elif length < 0:
            length = MAX_LENGTH  # avoid infinite loop
        return length
        
    length = adjust_length_to_model(length=length, max_sequence_length=model.config.max_position_embeddings)

    encoded_prompt = tokenizer.encode(prompt_text, add_special_tokens=False, return_tensors="pt")

    encoded_prompt = encoded_prompt.to(device)

    output_sequences = model.generate(
            input_ids=encoded_prompt,
            max_length=length + len(encoded_prompt[0]),
            temperature=temperature,
            top_k=k,
            top_p=p,
            repetition_penalty=repetition_penalty,
            do_sample=True,
            num_return_sequences=num_return_sequences,
        )

    if len(output_sequences.shape) > 2:
        output_sequences.squeeze_()

    generated_sequences = []

    for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
        #print("=== GENERATED SEQUENCE {} ===".format(generated_sequence_idx + 1))
        generated_sequence = generated_sequence.tolist()

        # Decode text
        text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)

        # Remove all text after the stop token
        text = text[: text.find(stop_token) if stop_token else None]

        # Add the prompt at the beginning of the sequence. Remove the excess text that was used for pre-processing
        total_sequence = (
            prompt_text + text[len(tokenizer.decode(encoded_prompt[0], clean_up_tokenization_spaces=True)) :]
        )

        generated_sequences.append(total_sequence)
    return generated_sequences


In [ ]:
model, tokenizer = get_model_tokenizer(weights_dir, device = 'cuda')

Loading Model ...
Model Loaded ...


In [ ]:
temperature = 1.0
k=400
p=0.9
repetition_penalty = 1.0
num_return_sequences = 5
length = 500
stop_token = '|EndOfText|'
prompt_text = "Jane is"

In [ ]:
%%time
generate_messages(
    model,
    tokenizer,
    prompt_text,
    stop_token,
    length,
    num_return_sequences,
    temperature = temperature,
    k=k,
    p=p,
    repetition_penalty = repetition_penalty
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


CPU times: user 7.06 s, sys: 32.1 ms, total: 7.09 s
Wall time: 7.12 s


['Jane is familiar with the process of the secretarial work at The Library of the Library of Congress (LCUSA) from which he derived his name was given him by me as a disciple of a teacher who received the title of Assistant Professor of Literature at the University of Virginia, at a time when the style of the subject has lost a great deal of its literary effect.\n\nThat was probably the first such title which came to my attention in a remark which I made about Mrs Sabel de Vilghery, my mother. We have very few persons of the female generation who have lived to this day, but perhaps a very few who have yet to appear to that age or more who have moved gradually to that fashion for whose sake others have all become mere legends from the flesh to this day and that which they may be called by their old names—towards the excellency of my mother—not as allotiable as those of those who possess a trifling later date were they always supposed to be those of a more young age for whom the last vol

# 4. GPT-2

In [ ]:
import torch
import torch.nn as nn
from torchtext.data.utils import get_tokenizer

In [ ]:
def read_txt(txt_path):
  with open(txt_path, 'r') as f:
    txt_string = f.readlines()
  return txt_string

corpus_string = read_txt("/content/drive/MyDrive/2022/NLP/test-data.txt")
corpus_string = "".join(corpus_string).replace('\n', ' ')

In [ ]:
corpus_string[:100]

' the forethought here lie buried my thgs which if read with patice may show the strge meg of beg bla'

In [ ]:
tokenizer = get_tokenizer('basic_english')

tokenizer(corpus_string)[:20]


from torchtext.vocab import build_vocab_from_iterator
vocab = build_vocab_from_iterator([tokenizer(corpus_string)])

In [ ]:
from torchtext.vocab import build_vocab_from_iterator
vocab = build_vocab_from_iterator([tokenizer(corpus_string)], specials=['<unk>'])

In [ ]:
sequence_of_words = tokenizer(corpus_string)[:20]
print(sequence_of_words)
vocab(sequence_of_words)

['the', 'forethought', 'here', 'lie', 'buried', 'my', 'thgs', 'which', 'if', 'read', 'with', 'patice', 'may', 'show', 'the', 'strge', 'meg', 'of', 'beg', 'black']


[1,
 7696,
 114,
 1124,
 1563,
 39,
 146,
 21,
 52,
 293,
 10,
 2022,
 88,
 388,
 1,
 328,
 497,
 3,
 69,
 158]

In [ ]:
word_emb = nn.Embedding(len(vocab), 4)

sentence_in_wrd_idx = vocab(sequence_of_words)
sentence_in_wrd_idx = torch.LongTensor(sentence_in_wrd_idx)

sentence_in_wrd_idx, word_emb(sentence_in_wrd_idx)

(tensor([   1, 7696,  114, 1124, 1563,   39,  146,   21,   52,  293,   10, 2022,   88,  388,    1,  328,  497,    3,   69,  158]),
 tensor([[ 0.4832,  1.4899, -2.2859,  0.7132],
         [-1.7164,  0.2041, -3.1984,  0.8907],
         [ 1.8122,  0.9460,  0.1351,  1.3293],
         [ 1.3819,  0.4573,  0.3227, -1.7420],
         [ 1.1800,  1.0795,  0.2945, -0.5151],
         [ 0.1766, -0.2756, -0.6165, -0.4869],
         [-0.4799,  0.1775,  1.7938, -0.2011],
         [ 0.9035,  1.3564, -0.5290,  0.5278],
         [ 1.7629, -1.2098, -0.3466, -2.0153],
         [-1.4137,  1.1910,  1.9065, -1.1498],
         [-1.4100,  0.9022, -1.4979,  1.6631],
         [-0.5042, -0.7993, -0.1075, -1.0704],
         [ 0.3356,  0.3922, -0.7740, -0.6149],
         [-1.2027,  0.1614, -1.2534,  0.4984],
         [ 0.4832,  1.4899, -2.2859,  0.7132],
         [ 0.5996,  0.3218,  0.1164, -0.1269],
         [-1.6706, -0.4373,  1.7190, -0.4095],
         [ 0.4242, -0.6905, -0.7752,  1.2347],
         [-0.5833, -0.6

In [ ]:
from pathlib import Path

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install transformers==4.2.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 5.3 MB/s 
     |████████████████████████████████| 880 kB 55.4 MB/s 
     |████████████████████████████████| 2.9 MB 50.6 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=83aaaefa02736fe162b7b575b2ccb66eb0208f6b06a129a6b72707b33090f957
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.12.1
    Uninstalling tokenizers-0.12.1:
      Successfully uninstalled tokenizers-0.12.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.20.0.dev0
    Uninstalling transformers-4.20.0.dev0:
      Successfully uninstalled transformers-4.20.0.dev0


In [ ]:
!nvidia-smi

Tue Jun  7 14:11:11 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P0    33W /  70W |   2496MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!unzip '/content/archive.zip'

unzip:  cannot find or open /content/archive.zip, /content/archive.zip.zip or /content/archive.zip.ZIP.


In [ ]:
import torch
from torch.utils.data import Dataset
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilgpt2")

train_path = '/content/gdrive/MyDrive/2022/NLP/train-data'
test_path = '/content/gdrive/MyDrive/2022/NLP/test-data'

Downloading:   0%|          | 0.00/762 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
from transformers import TextDataset,DataCollatorForLanguageModeling

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,
Token indices sequence length is longer than the specified maximum sequence length for this model (445556 > 1024). Running this sequence through the model will result in indexing errors


In [ ]:
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("distilgpt2")


training_args = TrainingArguments(
    output_dir="./gpt2-gerchef", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:925: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Downloading:   0%|          | 0.00/353M [00:00<?, ?B/s]

In [ ]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=327, training_loss=5.735854775898318, metrics={'train_runtime': 234.7814, 'train_samples_per_second': 1.393, 'total_flos': 656768481361920, 'epoch': 3.0})

In [ ]:
# train된 코드로 text generate 하는 것